In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os

In [26]:
load_dotenv()
output_csv = os.getenv("INPUT_CSV")
df = pd.read_csv(output_csv)

In [27]:
video_ids = df['video_id'].unique()

X_list = []
y_list = []

for vid in video_ids:
    video_df = df[df['video_id'] == vid].sort_values('frame')
    
    features = video_df.drop(columns=['video_id', 'frame', 'label']).values
    
    if features.shape[0] == 90:
        X_list.append(features)
        y_list.append(video_df['label'].iloc[0]) 

X = np.array(X_list)
y = np.array(y_list)
print(X.shape, y.shape)

(960, 90, 26) (960,)


In [ ]:
num_samples, timesteps, num_features = X.shape
X_reshaped = X.reshape(-1, num_features)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_reshaped)

X_scaled = X_scaled.reshape(num_samples, timesteps, num_features)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

model = Sequential([
    Masking(mask_value=0., input_shape=(timesteps, num_features)), 
    LSTM(128, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')  # klasyfikacja wieloklasowa
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
# history = model.fit(
#     X_train, y_train,
#     validation_split=0.2,
#     epochs=30,
#     batch_size=8,
#     verbose=1
# )

c:\Users\skrza\Documents\GitHub\ENGINEERING-WORK\.venv\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_5 (Masking)             │ (None, 90, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        23,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,607 (100.03 KB)

 Trainable params: 25,607 (100.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.2105 - loss: 1.9274 - val_accuracy: 0.3377 - val_loss: 1.7343
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.1926 - loss: 1.7217 - val_accuracy: 0.2597 - val_loss: 1.5237
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2736 - loss: 1.5333 - val_accuracy: 0.2532 - val_loss: 1.9239
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2200 - loss: 1.8859 - val_accuracy: 0.2597 - val_loss: 1.6758
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.2564 - loss: 1.7149 - val_accuracy: 0.2403 - val_loss: 1.6608
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2669 - loss: 1.6339 - val_accuracy: 0.2143 - val_loss: 1.5158
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.2906 - loss: 1.4543 - val_accuracy: 0.2597 - val_loss: 1.4603
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2753 - loss: 1.6384 - val_accuracy: 0.

In [32]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {acc:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3435 - loss: 1.3936
Test loss: 1.4025
Test accuracy: 0.3281


In [17]:
model.save("my_model.keras")

In [18]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_2 (Masking)             │ (None, 90, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        23,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,823 (300.09 KB)

 Trainable params: 25,607 (100.03 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 51,216 (200.07 KB)

In [21]:
y_pred = model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [24]:
y_pred[0]

array([4.9300104e-02, 5.4805456e-05, 5.8371330e-05, 1.4907702e-02,
       1.6970800e-03, 9.3396133e-01, 2.0697245e-05], dtype=float32)